# Acquisition des votes des député.e.s et sénateur.rice.s

N'ayant pas accès à la page https://git.regardscitoyens.org/regardscitoyens/sprint-scrutins/-/tree/master, il faut trouver un autre moyen d'obtenir les votes des débuté.e.s et sénateur.rice.s pour appliquer la méthode de vérification papier.

Par souci de simplicité, nous dirons députés pour député.e.s et sénateurs pour sénateur.rice.s.

Nous commencerons dans ce fichier par l'extraction des données liées au Sénat.
Pour cela et après quelques tests empiriques, il a été décider d'utiliser une méthode de scraping à l'aide de la bibliothèque Beautifulsoup.

Description de la méthode est la suivante :
1. Etudier la structure d'une page internet d'un scrutin sur le site du Sénat
2. Extraire les informations pertinentes de la page et les intégrer à un fichier csv
3. Scaler le procéder pour scraper l'ensemble des pages scrutins pour la période 11 octobre 2022 - 30 mars 2023*

*le choix de s'arrêter à la période 11 octobre 2022 - 30 mars 2023 est lié à une volonté de sobriété et aux limitations techniques liées au matériel à notre disposition.

### 1. Structure de la page internet du Sénat

On remarque que les noms des députés sont donnés dans des balises < a > après des balise < b > "Ont voté pour", "Ont voté contre", "Abstentions", "N'ont pas pris part au vote".

Nous allons utiliser ces balises pour récupérer les données de votes nominatives.

In [23]:
import requests
from bs4 import BeautifulSoup
import re

# URL de la page web à scraper
url = "https://www.senat.fr/scrutin-public/2022/scr2022-257.html"

# Récupération du contenu HTML de la page web
response = requests.get(url)
html_content = response.content

# Analyse du contenu HTML avec BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Affichage du contenu HTML
print(soup.prettify())

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" 
"http://www.w3.org/TR/html4/loose.dtd">
<!--[if IE 6]><html class="msie6"><![endif]-->
<!--[if IE 7]><html class="msie7"><![endif]-->
<!--[if gte IE 8]><html class="msie8"><![endif]-->
<!--[if !IE]><!-->
<html lang="fr">
 <!-- <![endif]-->
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   Scrutin n° 257 - séance du 21 mars 2023
  </title>
  <script src="/tarteaucitron/tarteaucitron.js" type="text/javascript">
  </script>
  <link href="/fileadmin/templates/css/base.css" rel="stylesheet" type="text/css"/>
  <link href="/fileadmin/templates/css/base_imports.css" rel="stylesheet" type="text/css"/>
  <link href="/css/dosleg.css" rel="stylesheet" type="text/css"/>
  <script type="text/javascript">
   var hasJS = function() {
			var htmlTag = document.getElementsByTagName('html')[0];
			htmlTag.className = (htmlTag.className + ' ' || '') + 'hasJS';
		}();
  </script>
  <!--#template virtu

### 2. Extraction des informations

La première étape de l'extraction des données consiste à extraire un échantillon de la page étudiée et à l'afficher dans Jupyter pour vérifier que cette méthode fonctionne.

In [22]:
import requests
from bs4 import BeautifulSoup

# URL de la page à scraper
url = "https://www.senat.fr/scrutin-public/2022/scr2022-257.html"

# Envoi d'une requête GET pour récupérer le contenu de la page
response = requests.get(url)

# Analyse du contenu HTML avec BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Extraction de l'information souhaitée
title_tag = soup.find('title')
scrutin_info = title_tag.text.strip()

# Affichage de l'information récupérée
print(scrutin_info)

# Récupération des balises <a> après la balise <b> "Ont voté pour"
ont_vote_pour = soup.find('b', text='Ont voté pour')
senateurs_pour = [a.text.strip().replace('\n', ' ') for a in ont_vote_pour.find_next('table').find_all('a')]

# Affichage des noms des sénateurs
for senateur in senateurs_pour:
    print(senateur)

# Récupération des balises <a> après la balise <b> "Ont voté contre"
ont_vote_contre = soup.find('b', text='Ont voté contre')
senateurs_contre = [a.text.strip().replace('\n', ' ') for a in ont_vote_contre.find_next('table').find_all('a')]

# Affichage des noms des sénateurs
for senateur in senateurs_contre:
    print(senateur)
    
# Récupération des balises <a> après la balise <b> "Abstentions"
abstentions = soup.find('b', text='Abstentions')
senateurs_abstentionistes = [a.text.strip().replace('\n', ' ') for a in abstentions.find_next('table').find_all('a')]

# Affichage des noms des sénateurs
for senateur in senateurs_abstentionistes:
    print(senateur)

# Récupération des balises <a> après la balise <b> "N'ont pas pris part au vote"
pas_pris_part = soup.find('b', text="N'ont pas pris part au vote")
senateurs_pas_pris_part = [a.text.strip().replace('\n', ' ') for a in pas_pris_part.find_next('table').find_all('a')]

# Affichage des noms des sénateurs
for senateur in senateurs_pas_pris_part:
    print(senateur)

Scrutin n° 257 - séance du 21 mars 2023
Pascal Allizard
Pierre Cuypers
Jean-Raymond Hugonet
Louis-Jean de Nicolaÿ
Jean-Claude Anglars
Laure Darcos
Jean-François Husson
Sylviane Noël
Jean-Michel Arnaud
Mathieu Darnaud
Corinne Imbert
Claude Nougein
Serge Babary
Marc-Philippe Daubresse
Annick Jacquemet
Olivier Paccaud
Jean Bacci
Jean-Pierre Decool
Micheline Jacques
Jean-Jacques Panunzi
Jean-Pierre Bansard
Vincent Delahaye
Jean-Marie Janssens
Vanina Paoli-Gagin
Philippe Bas
Bernard Delcros
Else Joseph
Philippe Paul
Jérôme Bascher
Véronique Del Fabro
Muriel Jourda
Cyril Pellevat
Arnaud Bazin
Annie Delmont-Koropoulis
Alain Joyandet
Philippe Pemezec
Arnaud de Belenet
Patricia Demas
Roger Karoutchi
Cédric Perrin
Bruno Belin
Stéphane Demilly
Claude Kern
Évelyne Perrot
Nadine Bellurot
Catherine Deroche
Christian Klinger
Annick Petrus
Catherine Belrhiti
Chantal Deseyne
Sonia de La Provôté
Stéphane Piednoir
Martine Berthet
Yves Détraigne
Laurent Lafon
Kristina Pluchet
Annick Billon
Brigitte Devésa

La deuxième étape de cette partie extraction de la donnée d'une seule page consite à essayer d'extraire un scrutin entier vers un fichier CSV.

In [30]:
import csv
import requests
from bs4 import BeautifulSoup

# URL de la page à scraper
url = "https://www.senat.fr/scrutin-public/2022/scr2022-257.html"

# Envoi d'une requête GET pour récupérer le contenu de la page
response = requests.get(url)

# Analyse du contenu HTML avec BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Extraction de l'information souhaitée
title_tag = soup.find('title')
scrutin_info = title_tag.text.strip().split(' - séance du ')

# Récupération du numéro de scrutin
scrutin_num = scrutin_info[0].split(' ')[-1]

# Récupération de la date de séance
date_seance = scrutin_info[1]

# Récupération des balises <a> après la balise <b> "Ont voté pour"
ont_vote_pour = soup.find('b', text='Ont voté pour')
senateurs_pour = [(senateur.text.strip().replace('\n', ' '), "pour") for senateur in ont_vote_pour.find_next('table').find_all('a')]

# Récupération des balises <a> après la balise <b> "Ont voté contre"
ont_vote_contre = soup.find('b', text='Ont voté contre')
senateurs_contre = [(senateur.text.strip().replace('\n', ' '), "contre") for senateur in ont_vote_contre.find_next('table').find_all('a')]

# Récupération des balises <a> après la balise <b> "Abstentions"
abstentions = soup.find('b', text='Abstentions')
senateurs_abstentionistes = [(senateur.text.strip().replace('\n', ' '), "abstention") for senateur in abstentions.find_next('table').find_all('a')]

# Récupération des balises <a> après la balise <b> "N'ont pas pris part au vote"
pas_pris_part = soup.find('b', text="N'ont pas pris part au vote")
senateurs_pas_pris_part = [(senateur.text.strip().replace('\n', ' '), "n'a pas pris part au vote") for senateur in pas_pris_part.find_next('table').find_all('a')]

# Création d'une liste avec toutes les informations
informations = [(scrutin_num, date_seance, senateur[0], senateur[1]) for senateur in (senateurs_pour + senateurs_contre + senateurs_abstentionistes + senateurs_pas_pris_part)]

# Enregistrement des informations dans un fichier CSV
with open('informations3.csv', mode='w', newline='', encoding='ISO-8859-1') as file:
    writer = csv.writer(file)
    writer.writerow(['scrutin_num', 'date_seance', 'senateur', 'vote'])
    for info in informations:
        writer.writerow(info)


### 3. Passage à l'échelle

Comme nous avons réussi à extraire de la page internet d'un scrutin, un fichier CSV qui contient les informations qui nous intéressent ('numéro du scrutin', 'date de la séance', 'prénom et nom du senateur', 'vote'), nous allons passer à l'échelle notre procéder.
La méthode utilisée ici est la création d'une classe avec Dataclasses, puis son utilisation au sein d'une boucle for.

In [36]:
import csv
import requests
from bs4 import BeautifulSoup
from dataclasses import dataclass

@dataclass
class Vote:
    scrutin_num: int
    date_seance: str
    senateur: str
    vote: str

class ScraperSenat:
    def __init__(self, url):
        self.url = url
        self.response = requests.get(url)
        self.soup = BeautifulSoup(self.response.text, 'html.parser')
        self.title_tag = self.soup.find('title')
        self.scrutin_info = self.title_tag.text.strip().split(' - séance du ')
        self.scrutin_num = int(self.scrutin_info[0].split(' ')[-1])
        self.date_seance = self.scrutin_info[1]
        self.senateurs_pour = self._get_senateurs_vote('Ont voté pour')
        self.senateurs_contre = self._get_senateurs_vote('Ont voté contre')
        self.senateurs_abstentionnistes = self._get_senateurs_vote('Abstentions')
        self.senateurs_pas_pris_part = self._get_senateurs_vote("N'ont pas pris part au vote")
        self.informations = self._get_informations()

    def _get_senateurs_vote(self, label):
        vote_label = self.soup.find('b', text=label)
        if vote_label is None:
            return []
        else:
            return [(senateur.text.strip().replace('\n', ' '), label.lower()) for senateur in vote_label.find_next('table').find_all('a')]

    def _get_informations(self):
        return [Vote(self.scrutin_num, self.date_seance, senateur[0], senateur[1]) for senateur in (self.senateurs_pour + self.senateurs_contre + self.senateurs_abstentionnistes + self.senateurs_pas_pris_part)]

    def write_to_csv(self, file_path):
        with open(file_path, mode='w', newline='', encoding='ISO-8859-1') as file:
            writer = csv.writer(file)
            writer.writerow(['scrutin_num', 'date_seance', 'senateur', 'vote'])
            for info in self.informations:
                writer.writerow(info.__dict__.values())


Au 30 mars 2023, date de l'extraction des données pour le Sénat, 257 scrutins ont déjà eu lieu en prenant pour point de départ le 11 octobre 2022.
Aussi, avant d'initier la boucle for, une vérification du bon fonctionnement* de la classe est nécessaire.
*Résultat juste et temps d'execution faible.

In [37]:
url = "https://www.senat.fr/scrutin-public/2022/scr2022-257.html"
scraper = ScraperSenat(url)
scraper.write_to_csv("resultats_scrutin.csv")

Passage à l'échelle avec l'ensemble des scrutins.

In [41]:
nb_scrutins = 257
with open("resultats_scrutin_concat.csv", mode='w', newline='', encoding='ISO-8859-1') as file:
    writer = csv.writer(file)
    writer.writerow(['scrutin_num', 'date_seance', 'senateur', 'vote'])
    for scrutin in range(1, nb_scrutins+1):
        url = f"https://www.senat.fr/scrutin-public/2022/scr2022-{scrutin}.html"
        scraper = ScraperSenat(url)
        for info in scraper.informations:
            writer.writerow(info.__dict__.values())
        print(f"scrutin {scrutin} concaténé")


scrutin 1 concaténé
scrutin 2 concaténé
scrutin 3 concaténé
scrutin 4 concaténé
scrutin 5 concaténé
scrutin 6 concaténé
scrutin 7 concaténé
scrutin 8 concaténé
scrutin 9 concaténé
scrutin 10 concaténé
scrutin 11 concaténé
scrutin 12 concaténé
scrutin 13 concaténé
scrutin 14 concaténé
scrutin 15 concaténé
scrutin 16 concaténé
scrutin 17 concaténé
scrutin 18 concaténé
scrutin 19 concaténé
scrutin 20 concaténé
scrutin 21 concaténé
scrutin 22 concaténé
scrutin 23 concaténé
scrutin 24 concaténé
scrutin 25 concaténé
scrutin 26 concaténé
scrutin 27 concaténé
scrutin 28 concaténé
scrutin 29 concaténé
scrutin 30 concaténé
scrutin 31 concaténé
scrutin 32 concaténé
scrutin 33 concaténé
scrutin 34 concaténé
scrutin 35 concaténé
scrutin 36 concaténé
scrutin 37 concaténé
scrutin 38 concaténé
scrutin 39 concaténé
scrutin 40 concaténé
scrutin 41 concaténé
scrutin 42 concaténé
scrutin 43 concaténé
scrutin 44 concaténé
scrutin 45 concaténé
scrutin 46 concaténé
scrutin 47 concaténé
scrutin 48 concaténé
s